# <div align="center">CP322-A Mini-Project 2: Machine Learning</div>
## <div align="center">Group 6</div>
### <div align="center">due on 12-Nov-2023 at 11:30 PM</div>

Imports:

In [1]:
import numpy as np
import gzip
# pip install -U scikit-learn

## Datasets

### Dataset 1 (20 Newsgroup): 

Use the default train subset (subset=‘train’, and remove=([‘headers’, ‘footers’, ‘quotes’]) in  sklearn.datasets) to train the models and report the final performance on the test subset.  note: you need to start with the text data and convert the text to feature vectors. Please refer to https://scikitlearn.org/stable/tutorial/text_analytics/working_with_text_data.html for a tutorial on the steps needed for this.

In [2]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

X = twenty_train.data
Y = twenty_train.target

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [3]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

print(twenty_train.target_names[twenty_train.target[0]])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
comp.graphics


### Dataset 2 (IMDB Reviews):

In [3]:
import gzip
import numpy as np

filename = "data/IMDB.gz"

downsampling_factor = 10

with gzip.open(filename, 'rb') as f:
    data = f.read()
    
print(data)

# lines = data.splitlines()
# max_columns = 0  # Initialize a variable to track the maximum number of columns in any row

# # Iterate through the lines and determine the maximum number of columns
# for i, line in enumerate(lines):
#     if i % downsampling_factor == 0:
#         row = line.split(',')
#         max_columns = max(max_columns, len(row))

# # Initialize a list to store the downsampled and padded rows
# downsampled_data_list = []

# # Iterate through the lines again, splitting and padding the rows
# for i, line in enumerate(lines):
#     if i % downsampling_factor == 0:
#         row = line.split(',')
#         # Pad with zeros if the row has fewer columns than the maximum
#         while len(row) < max_columns:
#             row.append('0')  # You can change '0' to another value if needed
#         downsampled_data_list.append(row)

# Convert the padded data list to a NumPy array

# print(downsampled_data_list)
# numpy_array = np.array(downsampled_data_list, dtype=float)

# print(numpy_array)


['aclImdb/\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000000755\x000016201\x000001750\x0000000000000\x0011601503240\x00010321\x00 5\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ustar  \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00amaas\x0

## Models

### Logistic Regression

In [15]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold


# Fetch the 20 newsgroups dataset
# newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Partiall dataset
# categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']


# Fetch the "train" subset of the data to be used
# newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Vectorizing the text from the given dataset using TF-IDF to get the matrix
data_vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf = data_vectorizer.fit_transform(newsgroups["data"])

# Defining the k-fold cross-validation with 5 folds measures to obtain the 5 groups
kFold = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

# Defining logistic regression model with 1000 interations
logistic_regression = LogisticRegression(max_iter=1000)

print(newsgroups["target_names"])
print(len(newsgroups["target"]))
print(len(newsgroups["data"]))


['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
11314
11314


### Decision Trees

In [16]:
from sklearn import tree
#borrow newsgroups from above cell

decision_tree_classifier = tree.DecisionTreeClassifier(criterion="gini", splitter="best") #other params that can be changed but defaults should be okay otherwise

decision_tree_classifier.fit(X_tfidf, newsgroups["target"])

DecisionTreeClassifier()

### Support Vector Machines

In [ ]:
from sklearn.svm import LinearSVC


svm_model = LinearSVC()

### Ada Boost

### Random Forest

## Validation

In [ ]:
from sklearn.model_selection import cross_val_score,  KFold
import pandas as pd


model_results = {
    "Model": [],
    "Dataset": [],
    "Accuracy (Mean)": [] 
}

kf = KFold(n_splits=5)

svm_scores = cross_val_score(svm_model, X_tfidf, Y, cv=kf)
svm_accuracy_mean = svm_scores.mean()

model_results["Model"].append("Support Vector Machines")
model_results["Dataset"].append("20 Newsgroup")
model_results["Accuracy (Mean)"].append(svm_accuracy_mean)

# DataFrame to report the performance of SVM
results_df = pd.DataFrame(model_results)

print(results_df)